In [3]:
from array_randomizer import PlateConfig, PlateVisualizer,PlateLayoutGenerator
import numpy as np
import matplotlib.pyplot as plt
# Example usage and demonstration

In [4]:
print("=" * 60)
print("PLATE LAYOUT GENERATION SYSTEM")
print("=" * 60)
print("\nDemonstration with two configuration modes:\n")

# ===== MODE 1: Using n_strains (generates default strain IDs) =====
print("-" * 60)
print("MODE 1: Using n_strains parameter")
print("-" * 60)

config1 = PlateConfig(
    n_strains=180,
    n_rows=8,
    n_columns=12,
    n_negatives=6,
    n_replicates=3,
    random_seed=42
)

print(f"  Strains: {config1.n_strains} (auto-generated IDs)")
print(f"  Plate format: {config1.n_rows} x {config1.n_columns}")
print(f"  Negatives per plate: {config1.n_negatives}")
print(f"  Replicates: {config1.n_replicates}")

generator1 = PlateLayoutGenerator(config1)
base_set1 = generator1.create_base_set()
print(f"\n  ✓ Generated {generator1.n_plates} plates with strain IDs: S0000, S0001, ..., S{config1.n_strains-1:04d}")

# ===== MODE 2: Using custom strain names =====
print("\n" + "-" * 60)
print("MODE 2: Using custom strain_names list")
print("-" * 60)

# Example: Create custom strain names
custom_strains = [
    "WT-Control",
    "Mutant-A1", "Mutant-A2", "Mutant-A3",
    "Mutant-B1", "Mutant-B2", "Mutant-B3",
    "KO-Gene1", "KO-Gene2", "KO-Gene3",
    "OE-Protein1", "OE-Protein2",
]
custom_strains += [f"Strain_{i:03d}" for i in range(13, 50)]  # Add more strains

config2 = PlateConfig(
    strain_names=custom_strains,  # Use custom names instead of n_strains
    n_rows=8,
    n_columns=12,
    n_negatives=4,
    n_replicates=2,
    random_seed=123
)

print(f"  Strains: {config2.n_strains} (from custom list)")
print(f"  First few strains: {custom_strains[:5]}")
print(f"  Plate format: {config2.n_rows} x {config2.n_columns}")
print(f"  Negatives per plate: {config2.n_negatives}")
print(f"  Replicates: {config2.n_replicates}")

generator2 = PlateLayoutGenerator(config2)
base_set2 = generator2.create_base_set()
results2 = generator2.generate_replicates()

print(f"\n  ✓ Generated {generator2.n_plates} plates with custom strain names")
print(f"  ✓ Generated {len(results2['replicates'])} replicates")

# Demonstrate strain lookup
print("\n  Strain Location Lookup Examples:")
test_strain = custom_strains[0]
locations = generator2.get_all_strain_locations(test_strain)
for rep_id, (plate_id, row, col) in locations.items():
    row_label = chr(65 + row)
    col_label = col + 1
    print(f"    {test_strain} in Rep {rep_id+1}: Plate {plate_id+1}, Well {row_label}{col_label}")

# ===== Continue with MODE 1 for full demonstration =====
print("\n" + "=" * 60)
print("Continuing full demonstration with MODE 1...")
print("=" * 60)

config = config1
generator = generator1
base_set = base_set1

print("\n" + "-" * 60)
print("STEP 2: Generating replicates with edge alternation...")
print("-" * 60)

results = generator.generate_replicates()

print(f"✓ Generated {len(results['replicates'])} replicates")
if results['metadata']['alternation_scores']:
    avg_score = np.mean(results['metadata']['alternation_scores'])
    print(f"✓ Average alternation score: {avg_score:.2%}")

# Create visualizer
print("\n" + "-" * 60)
print("STEP 3: Creating visualizations...")
print("-" * 60)

viz = PlateVisualizer(generator)

# Example visualizations
print("\nGenerating example visualizations:")
print("  1. Plate layout for Replicate 1, Plate 1")
fig1, _ = viz.visualize_plate_layout(0, 0)
plt.savefig('plate_layout_rep1_plate1.png', dpi=150, bbox_inches='tight')
print("     ✓ Saved: plate_layout_rep1_plate1.png")

if config.n_replicates > 1:
    print("  2. Edge alternation comparison (Rep 1 vs Rep 2)")
    fig2 = viz.visualize_edge_alternation(0, 1, 0)
    plt.savefig('edge_alternation_comparison.png', dpi=150, bbox_inches='tight')
    print("     ✓ Saved: edge_alternation_comparison.png")

print("  3. Negative distribution heatmap")
fig3 = viz.visualize_negative_distribution()
plt.savefig('negative_distribution.png', dpi=150, bbox_inches='tight')
print("     ✓ Saved: negative_distribution.png")

print("  4. Statistical summary dashboard")
fig4 = viz.visualize_summary()
plt.savefig('summary_dashboard.png', dpi=150, bbox_inches='tight')
print("     ✓ Saved: summary_dashboard.png")

# Example: Track specific strain
print("  5. Strain S0042 across all replicates")
fig5 = viz.visualize_all_replicates('S0042')
plt.savefig('strain_S0042_replicates.png', dpi=150, bbox_inches='tight')
print("     ✓ Saved: strain_S0042_replicates.png")

# Verification checklist
print("\n" + "=" * 60)
print("VERIFICATION CHECKLIST")
print("=" * 60)

# Check negative uniqueness
all_unique = True
for plate_id in range(generator.n_plates):
    positions_per_rep = []
    for rep_id in range(config.n_replicates):
        pos_set = set(results['metadata']['negative_positions'][rep_id][plate_id])
        positions_per_rep.append(pos_set)

    for i in range(len(positions_per_rep) - 1):
        if positions_per_rep[i] == positions_per_rep[i + 1]:
            all_unique = False
            break

print(f"✓ Negative uniqueness: {'PASS' if all_unique else 'FAIL'}")

# Check strain consistency
strain_consistent = True
for rep_id in range(config.n_replicates):
    for plate_id in range(generator.n_plates):
        strains_base = set()
        strains_rep = set()

        for row in range(config.n_rows):
            for col in range(config.n_columns):
                cell_base = base_set[plate_id, row, col]
                cell_rep = results['replicates'][rep_id][plate_id, row, col]

                if cell_base is not None and cell_base != "negative":
                    strains_base.add(cell_base)
                if cell_rep is not None and cell_rep != "negative":
                    strains_rep.add(cell_rep)

        if strains_base != strains_rep:
            strain_consistent = False
            break

print(f"✓ Strain consistency: {'PASS' if strain_consistent else 'FAIL'}")

# Check edge alternation
if results['metadata']['alternation_scores']:
    avg_score = np.mean(results['metadata']['alternation_scores'])
    alternation_pass = avg_score > 0.3  # At least 30% alternation
    print(f"✓ Edge alternation (avg {avg_score:.2%}): {'PASS' if alternation_pass else 'FAIL'}")

# Check randomization
if config.n_replicates > 1:
    identical = True
    for plate_id in range(generator.n_plates):
        if not np.array_equal(results['replicates'][0][plate_id],
                             results['replicates'][1][plate_id]):
            identical = False
            break
    randomization_pass = not identical
    print(f"✓ Randomization: {'PASS' if randomization_pass else 'FAIL'}")

print("\n" + "=" * 60)
print("GENERATION COMPLETE")
print("=" * 60)
print("\nAll visualizations saved. Review the images to validate the layout.")
print(f"\nRandom seed used: {config.random_seed if config.random_seed is not None else 'None (random)'}")
print("  - Use the same seed to reproduce identical layouts")
print("  - Set random_seed=None for different layouts each run")
print("\n" + "=" * 60)
print("USAGE EXAMPLES")
print("=" * 60)
print("\n1. With numeric strain count (auto-generated IDs):")
print("   config = PlateConfig(n_strains=100, n_rows=8, n_columns=12,")
print("                        n_negatives=6, n_replicates=3)")
print("\n2. With custom strain names:")
print("   my_strains = ['WT-Control', 'Mutant-A', 'Mutant-B', ...]")
print("   config = PlateConfig(strain_names=my_strains, n_rows=8,")
print("                        n_columns=12, n_negatives=6, n_replicates=3)")
print("\n3. Basic workflow:")
print("   generator = PlateLayoutGenerator(config)")
print("   generator.create_base_set()")
print("   results = generator.generate_replicates()")
print("\n4. Find strain locations:")
print("   location = generator.get_strain_location('WT-Control', replicate_id=0)")
print("   all_locations = generator.get_all_strain_locations('WT-Control')")
print("\n5. Visualize:")
print("   viz = PlateVisualizer(generator)")
print("   viz.visualize_plate_layout(replicate_id=0, plate_id=0)")
print("   viz.visualize_summary()")
print("\nFor custom analysis, access:")
print("  - generator.replicates[rep_id][plate_id] for plate arrays")
print("  - generator.metadata for scores, positions, and strain names")
print("  - generator.metadata['strain_names'] for the strain list used")

plt.show()

PLATE LAYOUT GENERATION SYSTEM

Demonstration with two configuration modes:

------------------------------------------------------------
MODE 1: Using n_strains parameter
------------------------------------------------------------
  Strains: 180 (auto-generated IDs)
  Plate format: 8 x 12
  Negatives per plate: 6
  Replicates: 3

  ✓ Generated 2 plates with strain IDs: S0000, S0001, ..., S0179

------------------------------------------------------------
MODE 2: Using custom strain_names list
------------------------------------------------------------
  Strains: 49 (from custom list)
  First few strains: ['WT-Control', 'Mutant-A1', 'Mutant-A2', 'Mutant-A3', 'Mutant-B1']
  Plate format: 8 x 12
  Negatives per plate: 4
  Replicates: 2


AttributeError: 'PlateLayoutGenerator' object has no attribute '_calculate_alternation_score'